# **MNIST CNN**

<모듈 임포트>

In [0]:
import torch
from torch import nn as nn
from torchvision import datasets
from torchvision import transforms

<GPU 확인>

In [10]:
torch.cuda.is_available()

True

<하이퍼 파라미터>

In [0]:
num_epochs = 5
batch_size = 64
learning_rate = 0.001
device = torch.device("cuda:0")

<데이터 불러오기>

In [4]:
train_dataset  = datasets.MNIST(root='./data/',
                                train=True,
                                transform=transforms.ToTensor(),
                                download=True)
    
test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

train_loader  = torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=batch_size,
                                            shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 18610395.42it/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 298402.14it/s]                           
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 4939308.67it/s]                           
8192it [00:00, 122717.30it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


<모델 정의>

In [0]:
class CNNMNIST(nn.Module):
    def __init__(self):
        super(CNNMNIST, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))

        self.fc = nn.Linear(32 * 7 * 7, 10)

    def forward(self, x):
        o = self.layer1(x)
        o = self.layer2(o)
        o = o.view(o.size(0), -1)
        o = self.fc(o)
        return o

<모델 선언>

In [0]:
model = CNNMNIST().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.99)
criterion = nn.CrossEntropyLoss()

<모델 학습>

In [15]:
model.train()
for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.369257
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.115528
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.299690
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.038308
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.065009
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.085971
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.091807
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.023310
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.157115
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.006944
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.229434
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.006503
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.113062
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.118013
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.014902
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.025488
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.031280
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.056414
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.033571
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.020298
Train Epoch:

<모델 검증>

In [16]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():    
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)        
        test_loss += criterion(output, target).item() # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))


Test set: Average loss: 0.0005, Accuracy: 9889/10000 (99%)

